In [4]:
 # importing necessary libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score,KFold
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm


In [5]:
 # Reading the Csv File 
df = pd.read_csv( 'AMAZON_DATA.csv' )

### **DATA CLEANING**

In [6]:
 # getting the first and last 5 rows of the data set 
display(df.head())
display(df.tail())

 # Getting the number of columns and rows 
print(df.shape)

 # Getting the column names 
print(df.columns.tolist())

,Product_ID,Product_Name,Category,Product_Link,Discounted_Price,Actual_Price,Rating,Rating_Count,Reviewers,Reviews
0,NaN,Graco Extend2Fit Convertible Baby Car Seat,Baby Products$Car Seats & Accessories,https://www.amazon.com/Graco-Extend2Fit-Conver...,$149.99,List Price: $199.99,4.8 out of 5,"70,954 global ratings",Amazon Customer|Amazon Customer|Chiara Flatter...,Versatile and Comfortable – A Top Choice for G...
1,B019EGMGR0,Graco Extend2Fit Convertible Baby Car Seat,Baby Products$Car Seats & Accessories,https://www.amazon.com/Graco-Extend2Fit-Conver...,$149.99,List Price: $199.99,4.8 out of 5,"70,954 global ratings",Amazon Customer|Amazon Customer|Chiara Flatter...,Versatile and Comfortable – A Top Choice for G...
2,B01LWMWAXV,King Size 4 Piece Sheet Set - Comfy Breathable...,Home & Kitchen$Bedding,https://www.amazon.com/King-Size-Sheet-Set-Bre...,$29.74,Typical price: $34.99,4.5 out of 5,"349,108 global ratings",Amazon Customer|Amazon Customer|John cromwell|...,Finally we BOTH can sleep comfy!|Luxurious Com...
3,B07XFC3MF3,MZOO Sleep Eye Mask for Men Women,Health & Household$Health Care,https://www.amazon.com/MZOO-Contoured-Sleeping...,$16.99,Typical price: $22.99,4.6 out of 5,"93,186 global ratings",Amazon Customer|Amazon Customer|Nithin Kumar|C...,Product Design|**A Nighttime Revelation: The M...
4,B08YYVRXLM,Momcozy Pregnancy Pillows for Sleeping,Baby Products$Pregnancy & Maternity,https://www.amazon.com/Momcozy-Pregnancy-Mater...,$36.99,List Price: $54.99,4.5 out of 5,"21,077 global ratings",Amazon Customer|Amazon Customer|Jammie|Nick|Ki...,Comfortable!|Not just for pregnancy|It’s cool ...


,Product_ID,Product_Name,Category,Product_Link,Discounted_Price,Actual_Price,Rating,Rating_Count,Reviewers,Reviews
2002,B09XM74VQL,maamgic 2 in 1 Mens Swim Trunks 7 Inch Swim Sh...,"Clothing, Shoes & Jewelry$Men",https://www.amazon.com/TP-Link-EAP610-Ultra-Sl...,NaN,NaN,4.7 out of 5,"1,416 global ratings",Amazon Customer|Amazon Customer|Nikki|Kyle Jon...,Boyfriend's Favorite Shorts|Very Comfy & Dries...
2003,B0B3T5BD2K,PINSPARK Womens Tennis Skirt Pleated Athletic ...,"Clothing, Shoes & Jewelry$Women",https://www.amazon.com/Portable-Professional-R...,NaN,NaN,4.6 out of 5,703 global ratings,Amazon Customer|Amazon Customer|SY|Rachel|Gric...,"Cute & Comfortable|So cute, great fit, flatter..."
2004,B08CX78LHL,NaN,NaN,https://www.amazon.com/Bedsure-Waterproof-Blan...,NaN,NaN,NaN,NaN,NaN,NaN
2005,B0BY8CGS1X,NaN,NaN,https://www.amazon.com/maamgic-Trunks-Shorts-S...,NaN,NaN,NaN,NaN,NaN,NaN
2006,B0CTSDF2KR,NaN,NaN,https://www.amazon.com/PINSPARK-Waisted-Pocket...,NaN,NaN,NaN,NaN,NaN,NaN


(2007, 10)
['Product_ID', 'Product_Name', 'Category', 'Product_Link', 'Discounted_Price', 'Actual_Price', 'Rating', 'Rating_Count', 'Reviewers', 'Reviews']


From the output we can see that the dataset contains 2,007 rows and 10 columns where the column names are 'Product_ID','Product_Name', 'Category', 'Product_Link', 'Discounted_Price', 'Actual_Price', 'Rating', 'Rating_Count', 'Reviewers' and'Reviews'.

Additionally, we identified the presence of null values within the dataset. These missing values will require appropriate handling, either by removing the affected entries or replacing them.

In [7]:
 # Checking for Null values 
print(df.isnull().sum())

Product_ID            1
Product_Name          3
Category              3
Product_Link          0
Discounted_Price    940
Actual_Price        931
Rating                9
Rating_Count          9
Reviewers             4
Reviews              17
dtype: int64


In [8]:
df.dropna( inplace = True )

 # Checking how many data is left 
print(df.shape)

(1049, 10)


After dropping the null value we can see that the dataset has remained 1049 rows 

In [9]:
 # Getting the data types of each columns 
print(df.dtypes) 

Product_ID          object
Product_Name        object
Category            object
Product_Link        object
Discounted_Price    object
Actual_Price        object
Rating              object
Rating_Count        object
Reviewers           object
Reviews             object
dtype: object


We can see that the columns have datatypes object and we therefore  need to convert specific columns, such as prices, ratings, and rating counts, into numeric formats. However before converting, it is important to replace any non-numeric characters such as commas, text, and whitespaces that may prevent a successful conversion to float or integer.

We will first review the unique values in the columns that require numeric conversion.

In [ ]:
 # Checking the different unique values in Actual_Price, Discounted_Price, Rating and Rating_count 
print(df['Actual_Price'].unique()) 
print(df['Discounted_Price'].unique())
print(df['Rating'].unique())
print(df['Rating_Count'].unique()) 

we can see that Actual_price contains unique values that contain texts Typical price , List price , comma , per  Count ,per Inch and dollar sign . While  Discounted Price contains $ and commas 

Rating_count contains the text global ratings and commas while ratings contain the text out of 5

In [11]:
 # Replacing the string values with empty string ''
df['Actual_Price'] = df['Actual_Price'].str.replace('Typical price','')
df['Actual_Price'] = df['Actual_Price'].str.replace('List Price','')
df['Actual_Price'] = df['Actual_Price'].str.replace('per Count','')
df['Actual_Price'] = df['Actual_Price'].str.replace('per count','')
df['Actual_Price'] = df['Actual_Price'].str.replace('per Inch','')
df['Actual_Price'] = df['Actual_Price'].str.replace('per Ounce','')
df['Actual_Price'] = df['Actual_Price'].str.replace('per Foot','')
df['Actual_Price'] = df['Actual_Price'].str.replace('per Sq Ft','')
df['Actual_Price'] = df['Actual_Price'].str.replace('per Fl Oz','')
df['Actual_Price'] = df['Actual_Price'].str.replace('per Feet','')
df['Actual_Price'] = df['Actual_Price'].str.replace('per oz','')
df['Actual_Price'] = df['Actual_Price'].str.replace(':','')
df['Actual_Price'] = df['Actual_Price'].str.replace(' ','')
df['Actual_Price'] = df['Actual_Price'].str.replace(',','')
df['Actual_Price'] = df['Actual_Price'].str.replace('$','')


df['Discounted_Price'] = df['Discounted_Price'].str.replace(',','')
df['Discounted_Price'] = df['Discounted_Price'].str.replace('$','')
df['Discounted_Price'] = df['Discounted_Price'].str.replace(' ','')

df['Rating'] = df['Rating'].str.replace('out of 5','')
df['Rating'] = df['Rating'].str.replace(' ','')

df['Rating_Count'] = df['Rating_Count'].str.replace('global ratings','')
df['Rating_Count'] = df['Rating_Count'].str.replace(',','')
df['Rating_Count'] = df['Rating_Count'].str.replace(' ','')

In [12]:
 # Changing the Datatypes to float or integers 
df['Actual_Price'] = df['Actual_Price'].astype('float') 
df['Discounted_Price'] = df['Discounted_Price'].astype('float')

df['Rating'] = df['Rating'].astype('float')
df['Rating_Count'] = df['Rating_Count'].astype('int')

 # Printing the Datatypes
print(df.dtypes) 


Product_ID           object
Product_Name         object
Category             object
Product_Link         object
Discounted_Price    float64
Actual_Price        float64
Rating              float64
Rating_Count          int64
Reviewers            object
Reviews              object
dtype: object


We now need to check for duplicated values if any and remove them 

In [13]:
print( df.duplicated().sum() )

0


In [14]:
 # Printing the categories column
display( df['Category'].head() )

1    Baby Products$Car Seats & Accessories
2                   Home & Kitchen$Bedding
3           Health & Household$Health Care
4      Baby Products$Pregnancy & Maternity
6                   Home & Kitchen$Bedding
Name: Category, dtype: object

Since there are no duplicated values there is no need to drop the duplicated values.

Looking at the category column we can see that the categories are concatenated into a single string, separated by a dollar sign ($) therefore we need to split the string at the dollar sign then assign each split value to its respective columns.

In [15]:
 # Splitting the Categories and expanding the split categories into their separate columns 
category_split = df['Category'].str.split( '$' , expand = True )

display( category_split.head() )

,0,1
1,Baby Products,Car Seats & Accessories
2,Home & Kitchen,Bedding
3,Health & Household,Health Care
4,Baby Products,Pregnancy & Maternity
6,Home & Kitchen,Bedding


we can see that the categories have been split into two columns now we need to rename the columns to category_1 , Category_2 

In [16]:
 # Renaming the Categories 
category_split = category_split.rename( columns = { 0 : 'category_1' , 1 : 'category_2'} )

display( category_split.head() )

,category_1,category_2
1,Baby Products,Car Seats & Accessories
2,Home & Kitchen,Bedding
3,Health & Household,Health Care
4,Baby Products,Pregnancy & Maternity
6,Home & Kitchen,Bedding


We will now remove the original 'Category' column from the DataFrame and replace it with the newly created columns from splitting the category string.

In [17]:
 # Dropping the category column
df.drop( columns = 'Category' , inplace = True )

 # Replacing with the split categories 
df['category_1'] = category_split['category_1']
df['category_2'] = category_split['category_2']

display( df.head() )

,Product_ID,Product_Name,Product_Link,Discounted_Price,Actual_Price,Rating,Rating_Count,Reviewers,Reviews,category_1,category_2
1,B019EGMGR0,Graco Extend2Fit Convertible Baby Car Seat,https://www.amazon.com/Graco-Extend2Fit-Conver...,149.99,199.99,4.8,70954,Amazon Customer|Amazon Customer|Chiara Flatter...,Versatile and Comfortable – A Top Choice for G...,Baby Products,Car Seats & Accessories
2,B01LWMWAXV,King Size 4 Piece Sheet Set - Comfy Breathable...,https://www.amazon.com/King-Size-Sheet-Set-Bre...,29.74,34.99,4.5,349108,Amazon Customer|Amazon Customer|John cromwell|...,Finally we BOTH can sleep comfy!|Luxurious Com...,Home & Kitchen,Bedding
3,B07XFC3MF3,MZOO Sleep Eye Mask for Men Women,https://www.amazon.com/MZOO-Contoured-Sleeping...,16.99,22.99,4.6,93186,Amazon Customer|Amazon Customer|Nithin Kumar|C...,Product Design|**A Nighttime Revelation: The M...,Health & Household,Health Care
4,B08YYVRXLM,Momcozy Pregnancy Pillows for Sleeping,https://www.amazon.com/Momcozy-Pregnancy-Mater...,36.99,54.99,4.5,21077,Amazon Customer|Amazon Customer|Jammie|Nick|Ki...,Comfortable!|Not just for pregnancy|It’s cool ...,Baby Products,Pregnancy & Maternity
6,B07F946VMJ,Extra Deep Queen Sheet Set - 6 Piece Breathabl...,https://www.amazon.com/Piece-Fitted-Sheet-Set-...,34.92,79.99,4.4,79532,Amazon Customer|Amazon Customer|Hayley|J.|Judy...,You won’t be disappointed|Can't go wrong with ...,Home & Kitchen,Bedding


In [18]:
 # checking how many different categories there are 
print( df['category_1'].value_counts() )
print( df['category_2'].value_counts() )

category_1
Home & Kitchen                  451
Electronics                     122
Tools & Home Improvement         74
Health & Household               70
Cell Phones & Accessories        62
Beauty & Personal Care           46
Sports & Outdoors                39
Baby Products                    35
Pet Supplies                     32
Patio, Lawn & Garden             30
Industrial & Scientific          18
Office Products                  18
Automotive                       15
Toys & Games                     13
Clothing, Shoes & Jewelry         5
Musical Instruments               5
Appliances                        4
Arts, Crafts & Sewing             3
Video Games                       3
Flash Player                      1
Mobility & Daily Living Aids      1
Grocery & Gourmet Food            1
Kitchen & Dining                  1
Name: count, dtype: int64
category_2
Bedding                           161
Home Décor Products                99
Computers & Accessories            62
Kitchen & 

From the output we can see the number of unique values in the categories columns where in category_1, Home & Kitchen have the highest number of products while in category_2, Beddings has the highest number of products.

We will now add a new column that contains th rating_score of poor , Average , Above Average and Excellent.

In [19]:
 # Creating an empty list
Rating_score = []

 # Looping through the rating column 
for score in df['Rating']:
    if score < 2.0:
        Rating_score.append('Poor')
    elif score < 3.0:
        Rating_score.append('Below Average')
    elif score < 4.0:
        Rating_score.append('Average')
    elif score < 5.0:
        Rating_score.append('Above Average')
    elif score == 5.0:
        Rating_score.append('Excellent')

 # Adding the rating score to the dataframe
df['Rating_Score'] = Rating_score

display(df.head())

,Product_ID,Product_Name,Product_Link,Discounted_Price,Actual_Price,Rating,Rating_Count,Reviewers,Reviews,category_1,category_2,Rating_Score
1,B019EGMGR0,Graco Extend2Fit Convertible Baby Car Seat,https://www.amazon.com/Graco-Extend2Fit-Conver...,149.99,199.99,4.8,70954,Amazon Customer|Amazon Customer|Chiara Flatter...,Versatile and Comfortable – A Top Choice for G...,Baby Products,Car Seats & Accessories,Above Average
2,B01LWMWAXV,King Size 4 Piece Sheet Set - Comfy Breathable...,https://www.amazon.com/King-Size-Sheet-Set-Bre...,29.74,34.99,4.5,349108,Amazon Customer|Amazon Customer|John cromwell|...,Finally we BOTH can sleep comfy!|Luxurious Com...,Home & Kitchen,Bedding,Above Average
3,B07XFC3MF3,MZOO Sleep Eye Mask for Men Women,https://www.amazon.com/MZOO-Contoured-Sleeping...,16.99,22.99,4.6,93186,Amazon Customer|Amazon Customer|Nithin Kumar|C...,Product Design|**A Nighttime Revelation: The M...,Health & Household,Health Care,Above Average
4,B08YYVRXLM,Momcozy Pregnancy Pillows for Sleeping,https://www.amazon.com/Momcozy-Pregnancy-Mater...,36.99,54.99,4.5,21077,Amazon Customer|Amazon Customer|Jammie|Nick|Ki...,Comfortable!|Not just for pregnancy|It’s cool ...,Baby Products,Pregnancy & Maternity,Above Average
6,B07F946VMJ,Extra Deep Queen Sheet Set - 6 Piece Breathabl...,https://www.amazon.com/Piece-Fitted-Sheet-Set-...,34.92,79.99,4.4,79532,Amazon Customer|Amazon Customer|Hayley|J.|Judy...,You won’t be disappointed|Can't go wrong with ...,Home & Kitchen,Bedding,Above Average


We will now check the number of different rating score 

In [20]:
 # Printing the number of unique values in the rating score columns
print(df['Rating_Score'].value_counts())

Rating_Score
Above Average    1039
Excellent          10
Name: count, dtype: int64


From the dataset we can see that most products have a rating of Above average while 10 products have a rating of Excellent and no product has a rating below Above Average  

We will now change the datatype of the Rating_Score to category and reorder the categories 

In [21]:
df['Rating_Score'] = df['Rating_Score'].astype('category')
df['Rating_Score'] = df['Rating_Score'].cat.reorder_categories(['Above Average','Excellent'] , ordered = True )

We will generate a new column, 'price_difference', which will represent the difference between the actual price and the discounted price. Additionally, we will create another column to capture the discount percentage.

In [22]:
 # Price Difference 
df['Price_difference'] = df['Actual_Price'] - df['Discounted_Price']

 # Discount percentage
df['Discount_Percentage'] = np.trunc((df['Price_difference'] / df['Actual_Price']) * 100)

display(df.head())

,Product_ID,Product_Name,Product_Link,Discounted_Price,Actual_Price,Rating,Rating_Count,Reviewers,Reviews,category_1,category_2,Rating_Score,Price_difference,Discount_Percentage
1,B019EGMGR0,Graco Extend2Fit Convertible Baby Car Seat,https://www.amazon.com/Graco-Extend2Fit-Conver...,149.99,199.99,4.8,70954,Amazon Customer|Amazon Customer|Chiara Flatter...,Versatile and Comfortable – A Top Choice for G...,Baby Products,Car Seats & Accessories,Above Average,50.00,25.0
2,B01LWMWAXV,King Size 4 Piece Sheet Set - Comfy Breathable...,https://www.amazon.com/King-Size-Sheet-Set-Bre...,29.74,34.99,4.5,349108,Amazon Customer|Amazon Customer|John cromwell|...,Finally we BOTH can sleep comfy!|Luxurious Com...,Home & Kitchen,Bedding,Above Average,5.25,15.0
3,B07XFC3MF3,MZOO Sleep Eye Mask for Men Women,https://www.amazon.com/MZOO-Contoured-Sleeping...,16.99,22.99,4.6,93186,Amazon Customer|Amazon Customer|Nithin Kumar|C...,Product Design|**A Nighttime Revelation: The M...,Health & Household,Health Care,Above Average,6.00,26.0
4,B08YYVRXLM,Momcozy Pregnancy Pillows for Sleeping,https://www.amazon.com/Momcozy-Pregnancy-Mater...,36.99,54.99,4.5,21077,Amazon Customer|Amazon Customer|Jammie|Nick|Ki...,Comfortable!|Not just for pregnancy|It’s cool ...,Baby Products,Pregnancy & Maternity,Above Average,18.00,32.0
6,B07F946VMJ,Extra Deep Queen Sheet Set - 6 Piece Breathabl...,https://www.amazon.com/Piece-Fitted-Sheet-Set-...,34.92,79.99,4.4,79532,Amazon Customer|Amazon Customer|Hayley|J.|Judy...,You won’t be disappointed|Can't go wrong with ...,Home & Kitchen,Bedding,Above Average,45.07,56.0


From the output we can see that some price differences are negative meaning that the actual price is less than the discounted price which is undesirable

This is due to the fact that some products were sold by per inch , per count or per foot we will therefore need to drop any product that has a negative price difference.

In [23]:
 # Getting the index where the price difference is -
indexes = df.index[ df['Price_difference'] < 0 ].tolist()

 # Getting the length of indexes 
print( len(indexes) )

40


There are 40 products that have a price difference that is less than 0 therefore these 40 products need to be dropped 

In [24]:
 # dropping the indexes 
for i in indexes:
    df.drop( i , inplace = True )
 # Reseting the indexes 
df.reset_index( inplace = True )

print( df.shape )

(1009, 15)


We can see that the number of rows has dropped to 1009

We will now split the data set into two dataframes 

In [25]:
df1 = df[['Product_ID','Product_Name', 'category_1','Product_Link', 'category_2', 'Discounted_Price', 'Actual_Price','Price_difference','Discount_Percentage', 'Rating', 'Rating_Count', 'Rating_Score','Reviews']]

df2 = df[['Reviewers']]

In [26]:
display( df2.head(3) ) 

,Reviewers
0,Amazon Customer|Amazon Customer|Chiara Flatter...
1,Amazon Customer|Amazon Customer|John cromwell|...
2,Amazon Customer|Amazon Customer|Nithin Kumar|C...


From the ouput we can see that  the reviewers  of a product are concatenated into a single  string that is separated by a | representing the next person a. We will therefore need to split the  reviewers where there is a |

In [27]:
reviewers = df2['Reviewers']

 # Splitting the reviewers  
reviewers_split = reviewers.str.split( '|' , expand = False )

 # Making it show onw reviewer and review per row
reviewers_exp = reviewers_split.explode()

 # Reseting the Index
reviewers = reviewers_exp.reset_index( drop = True )

 # Changing the Series into dataframes 
df2 = pd.DataFrame( reviewers )

print( df2.shape )


(14806, 1)


After splitting the data we can see that the reviewers dataframe(df2) has 14,806 rows 

In [28]:
 # saving the Dataframe as csv 
df1.to_csv('CLEANED_DATA.csv', index=False)
df2.to_csv('REVIEWERS_DATA.csv', index=False)